In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import numpy as np
import scipy.integrate as sci
import time 
import threading
import sys
import multiprocessing as multiproc
import datetime
# the function library
from projectlib import *

## Parameters

**Fehler der Parameter wird aus der Datei "(bessere) Optimierung" entnommen**

In [206]:
alpha = 0.18
beta = 0.08
d_alpha = 0.1
d_beta = 0.01
method = 'constant'

## Simulation 

In [208]:
class Data:
    def __init__(self, alpha, beta, dimension, name, commuters_day, N):
        "Dimension gives the dimension of the system, i.e. number of cells"
        # ------- TO DO ---------
        # Depending on the system this has to be adjusted
        self.dimension = dimension
        
        "Alpha describes the rate of infection"
        self.alpha = alpha # Dimensions day^(-1)

        "Beta describes the recovery rate"
        self.beta = beta # Dimensions day^(-1)

        "p is the probability to die from the disease"
        self.p = 2.64e-2
        
        """
        Commuters is an array describing the commuters from and to the different cells. 
        The structure is: The entries in column i are how many commute from i to the cell of the row.
                          Thus, the entries of the row i describe how many commute from different cells to cell i.
        """

        self.commuters = np.array([np.ones(self.dimension) for i in range(self.dimension)])
        "load the txt with given name"
        file = np.loadtxt(name, delimiter="\t")
        "fill commuters with the entries from the file. Note the break statement so it doesn't go to far"
        i = 0
        for row in file:
            if i >= self.dimension:
                break
            self.commuters[i] = np.asarray(row[0:self.dimension])
            i += 1
        
        "Part of day that commuters are in other cells. This is the same value for all cells"
        self.commuters_day = commuters_day

        "Array of population in every cell"
        # ------- TO DO ---------
        # figure out how to best do this
        self.N = N

        
        
    def commutersFrom(self, cfrom):
        """Function to extract the commuters from a cell

        Args:
            cfrom (integer): the cell 

         Returns:
            array: array containing commuters from cfrom
        """
        # ------- TO DO ---------
        # optimize the algorithm
        "Sorting out the necessary entries into array a."
        a = np.zeros(self.dimension)
        for i in range(self.dimension):
            a[i] = self.commuters[i][cfrom]
            
        return a 
    
    def commutersTo(self, cto):
        """Function to return an array with number of commuters coming to cto from other cells.

        Args:
            cto (integer): the cell to which the commuters travel

        Returns:
            array: array containing the commuters to cto.
        """
        return self.commuters[cto][:]
    

### Simulation Functions

In [209]:

def function_of_system(timestep, functions,  data, method, t0):
    """Function to calculate the time derivatives of the dynamic SIRD modell with commuters. This function uses multiple approachs for different variant of the system based on prior assumptions. 

        Args:
            functions (array): array with the functions that define the system of equations. Structure [S_1, ... , S_n, I_1, .... , I_n, R_1, ... , R_n, D_1, ... , D_n]
            timestep (float): time of timestep. t=1 is 1 day
            data (class): the class with the relevant data
            method (string): dictates what method is to be used. The choices are "constant", "heaviside"
            t0 (array): array with len = 3 filled with times at which the heaviside functions in the "heaviside" method switch. 
                Note: all values of t0 have to be between 0 and 1.

        Returns:
            array: array containing the derivatives of functions at time t.
    """
    # array with infected
    Infected = functions[data.dimension:2*data.dimension]

    # making array with the effective infected Ieff
    Ieff = np.array([effective_infected(data.commutersTo, data.commutersFrom, data.N, i, Infected, data.dimension) for i in range(data.dimension)])

    # initializing the return array with the time derivatives of functions
    dfuncdt = np.ones(data.dimension*4)

    "~~~~~~~~~~~~~~~~~ See the PDF / LaTeX for further explanation of the equations ~~~~~~~~~~~~~~~~~~~~~~~~"

    ## ------------------------------------
    # -     Without commuting effects    -
    # ------------------------------------

    if method == 'simple':

        for i in range(data.dimension):
            # see LaTeX for equations
            # dS/dt
            dfuncdt[i] =  - data.alpha *  functions[i] * Infected[i]

            # dI/dt \propto -dSdt
            dfuncdt[i + data.dimension] = - dfuncdt[i] - data.beta * Infected[i]

            # dR/dt
            dfuncdt[i + 2 * data.dimension] = (1 - data.p) * data.beta * Infected[i]

            # dD/dt
            dfuncdt[i + 3 * data.dimension] = data.p * data.beta * Infected[i] 


    # ------------------------------------
    # -      Constant Coefficients       -
    # ------------------------------------

    tt = 0.6 #time in home vertex

    if method == "constant":
            
        # the for loop to fill dfuncdt

        for i in range(data.dimension):
            # see LaTeX for equations
            # dS/dt
            dfuncdt[i] =  - tt * data.alpha *  functions[i] * functions[i + data.dimension] - (1 - tt) * data.alpha * functions[i] * Ieff[i]

            # dI/dt \propto -dSdt
            dfuncdt[i + data.dimension] = - dfuncdt[i] - data.beta * functions[i + data.dimension]

            # dR/dt
            dfuncdt[i + 2 * data.dimension] = (1 - data.p) * data.beta * functions[i + data.dimension]

            # dD/dt
            dfuncdt[i + 3 * data.dimension] = data.p * data.beta * functions[i + data.dimension]


    # ------------------------------------
    # -      Heaviside Coefficients      -
    # ------------------------------------


    elif method == "heaviside":


        # the for loop to fill dfuncdt
        for i in range(data.dimension):
            # see LaTeX for equations
            # dS/dt
            dfuncdt[i] = - data.alpha * (periodic_heaviside(timestep, t0[0]) + periodic_heaviside(timestep, t0[2]) - 1) * functions[i] * functions[i + data.dimension] - (periodic_heaviside(timestep, t0[1]) - periodic_heaviside(timestep, t0[2])) * data.alpha * functions[i] * Ieff[i]

            # dI/dt \propto -dSdt
            dfuncdt[i + data.dimension] = - dfuncdt[i] - data.beta * functions[i + data.dimension]

            # dR/dt
            dfuncdt[i + 2 * data.dimension] = (1 - data.p) * data.beta * functions[i + data.dimension]

            # dD/dt
            dfuncdt[i + 3 * data.dimension] = data.p * data.beta * functions[i + data.dimension]

    
    
    
    
    
    
    
    # returning the derivatives at time t
    return dfuncdt



# Function for using scipy.integrate.odeint, as the arguments are switched
def system_function(functions, timestep, data, method, t0):
    return function_of_system(timestep, functions, data, method, t0)




### Initial Values

In [210]:
dimension = number_of_Lks
M =  region_setup(38)[0]
N = import_rki_data(M, 7)[2]
#N_ enthält Anzahl der Bevölkerung der Landkreis
RKI_data = np.load("Internal Data/timeline.npy")
I_RKI = np.array([RKI_data[i][1] for i in range(38)])
#I_RKI[Landkreis][Tag]

In [211]:
commuters = np.array([np.ones(dimension) for i in range(dimension)])
file = np.loadtxt("Pendler.txt", delimiter="\t")
"fill commuters with the entries from the file. Note the break statement so it doesn't go to far"
i = 0
for row in file:
    if i >= dimension:
        break
    commuters[i] = np.asarray(row[0:dimension])
    i += 1

In [212]:
b = initial_compartment_distribution(38, "2020/07/24")
initCond38 = np.zeros(38*4)
for i in range(38):
        initCond38[i] = b[i][0]
        initCond38[i + 38] = b[i][1]
        initCond38[i + 38*2] = b[i][2]
        initCond38[i + 38*3] = b[i][3]

simulation001 = Data(alpha, beta, dimension, "Pendler.txt", 0.6, N)
#t = np.linspace(0, 100, 10000)
t = np.linspace(0,100, 100)
tzero = np.array([0, 0.4, 0.8])
sol = sci.solve_ivp(fun=function_of_system, t_span=(0, 100),\
              y0=initCond38, method="RK45", args=(simulation001, method, tzero), t_eval=t, dense_output=True)
params = [alpha, beta, 2.64e-2]
state = sol.y
solt = sol.t

### Initial uncertainty

In [223]:
number_of_Lks = 38

Sigma_state = initCond38*0.01
Sigma_params = np.array([d_alpha,d_beta,0])
Sigma = np.diag(np.concatenate((Sigma_state,Sigma_params), axis=0, out=None, dtype=None, casting="same_kind"))


## Jacobian

In [224]:
def commutersFrom(cfrom):
    """Function to extract the commuters from a cell

    Args:
        cfrom (integer): the cell 

     Returns:
        array: array containing commuters from cfrom
    """

    "Sorting out the necessary entries into array a."
    a = np.zeros(dimension)
    for i in range(dimension):
        a[i] = commuters[i][cfrom]
            
    return a 
    
def commutersTo(cto):
    """Function to return an array with number of commuters coming to cto from other cells.

    Args:
        cto (integer): the cell to which the commuters travel

    Returns:
        array: array containing the commuters to cto.
    """
    return commuters[cto][:]
       


def I_eff(infected, i, t = 0):
    """Function for effective infected

    Args:
        infected (array): array with infected
        i (int): what cell
        t (int): time in days (counted from simmulation start)

    Returns:
        float: effective population in cell i
    """
    return effective_infected(commutersTo, commutersFrom, N, i, infected, dimension)

def N_eff(i):
    """Effective population

    Args:
        i (int): what cell
        t (int): time in days (counted from simmulation start)

    Returns:
        float: effective population in cell i
    """
    return N[i] - np.sum(commutersFrom(i))

def P(i,j,t = 0):
    return commuters[j][i]


In [225]:
def delta_kronecker(i, j):
    if i == j:
        return 1
    else:
        return 0

def ds(state, number_of_Lks, method, params, i, t):
    """
    calculates dS/dt im Landkreis i zum Zeitpunkt t
    method ist Vektor mit zwei Einträgen 0 ist SIGMA1_1 ist SIGMA_2
    """
    alpha = params[0]
    beta = params[1]
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    res = 0
    for j in range(number_of_Lks):
        res = res + P(i,j,t) * I_eff(I, j, t)
    return -method[0] * alpha * S[i] * I[i] - method[1] * alpha *\
           (N_eff(i)* S[i] *I_eff(I, i, t)  + S[i] * res) / N[i]

def dI(state, number_of_Lks, method, params, i, t):
    alpha = params[0]
    beta = params[1] 
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    
    res = 0
    for j in range(number_of_Lks):
        res = res + P(i,j,t) * I_eff(I, j, t)
    return - method[0] * alpha * S[i] * I[i] + method[1] * alpha *\
           (N_eff(i)* S[i] *I_eff(I, i, t)  + S[i] * res) / N[i] - beta * I[i]


def dR(state , number_of_Lks , params, i, t):
    beta = params[1]
    p = params[2]
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    return (1-p) * beta * I[i]

def dD(state , number_of_Lks , params, i, t):
    p = params[2]
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    return (p) * beta * I[i]


In [226]:
def Jacobian_SS(state, number_of_Lks, method, params, t):
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    n = number_of_Lks
    J_SS = np.identity(n)
    for i in range(n):
        J_SS[i][i] = ds(state, number_of_Lks, method, params, i, t)/S[i]
    return J_SS

def Jacobian_SI(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    alpha = params[0]
    beta = params[1]
    J_SI = np.identity(n)
    S = state[0:number_of_Lks,t]

    P = commuters
    Nr = np.array([N_eff(i) for i in range(n)])
    


    sumba = np.array([np.zeros(n) for i in range(n)])
    for i in range(n):
        for j in range(n):
                for l in range(n):
                    sumba[i, j] += P[l, i] * P[l, j] / (Nr[l] + sum(P[l, :]))
                sumba[i, j] *= 1.0 / N[i]


    for i in range(number_of_Lks):
        for j in range(number_of_Lks):
            if (i == j):
                J_SI[i, j] = ((method[1] * Nr[i]**2 / (N[i] * (Nr[i] + sum(P[i, :])))) + method[0]) * S[i]
            else:
                J_SI[i, j] =  method[1] * (P[i, j] * Nr[i] / (N[i] * (Nr[i] + sum(P[i, :]))) + P[j, i] * Nr[j] / (N[i] * (Nr[j] + sum(P[j, :]))) + sumba[i, j]) * S[i]
    
    return - alpha * J_SI




def Jacobian_RI(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    alpha = params[0]
    beta = params[1] 
    p = params[2]
    J_RI = np.identity(n)
    for i in range(n):
        for l in range(n):
            J_RI[i][l] = delta_kronecker(i, l) * (1 - p) * beta
    return J_RI

def Jacobian_DI(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    alpha = params[0]
    beta = params[1]
    p = params[2]
    J_DI = np.identity(n)
    for i in range(n):
        for l in range(n):
            J_DI[i][l] = delta_kronecker(i, l) * p * beta
    return J_DI

def Jacobian_IS(state, number_of_Lks, method, params, t):
    return -1* Jacobian_SS(state, number_of_Lks, method, params, t)

def Jacobian_II(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    beta = params[1]

    J_intermed = - Jacobian_SI(state, number_of_Lks, method, params, t)

    J_II = J_intermed - beta * np.identity(n)

    return J_II

In [227]:
def Jacobian(state, solt , number_of_Lks, method, params, t):
    time_of_day = solt[t] - np.floor(solt[t])
    if method=='constant':
        X = [0.6,0.4]
    elif method=='heaviside':
        if time_of_day >=tzero[0] and time_of_day <= tzero[1]:
            X = [1,0]
        elif time_of_day>tzero[1] and time_of_day <= tzero[2]:
            X = [0,1]
    else:
        print("invalid Method")
        X = [0.6,0.4]
    J_SS = Jacobian_SS(state, number_of_Lks, X, params, t)
    J_II = Jacobian_II(state, number_of_Lks, X, params, t)
    J_RI = Jacobian_RI(state, number_of_Lks, X, params, t)
    J_IS = Jacobian_IS(state, number_of_Lks, X, params, t)
    J_SI = Jacobian_SI(state, number_of_Lks, X, params, t)
    J_DI = Jacobian_DI(state, number_of_Lks, X, params, t)
    J_SR = np.identity(number_of_Lks)*0
    J_SD = np.identity(number_of_Lks) * 0
    J_IR = np.identity(number_of_Lks)*0
    J_ID = np.identity(number_of_Lks)*0
    J_RS = np.identity(number_of_Lks)*0
    J_DS = np.identity(number_of_Lks)*0
    J_RR = np.identity(number_of_Lks)*0
    J_DR = np.identity(number_of_Lks)*0
    J_RD = np.identity(number_of_Lks)*0
    J_DD = np.identity(number_of_Lks)*0
    Z = np.array([J_SS, J_SI, J_SR, J_SD])
    n = number_of_Lks
    J = np.identity(n*4)
    for i in range(n):
        for j in range(n):
            J[i][j]=J_SS[i][j]
            J[i+n][j]= J_SI[i][j]
            J[i + 2*n][j] = J_SR[i][j]
            J[i + 3 * n][j] = J_SD[i][j]
            J[i][j+ n] = J_IS[i][j]
            J[i][j + 2* n] = J_RS[i][j]
            J[i][j + 3 * n] = J_DS[i][j]
            J[i + n][j + n] = J_II[i][j]
            J[i + n][j + 2 * n] = J_RI[i][j]
            J[i + n][j + 3 * n] = J_DI[i][j]
            J[i + 2 * n][j + n] = J_IR[i][j]
            J[i + 2 * n][j + 2*n] = J_RR[i][j]
            J[i + 2 * n][j + 3 * n] = J_DR[i][j]
            J[i + 3 * n][j +  n] = J_ID[i][j]
            J[i + 3 * n][j + 2*n] = J_RD[i][j]
            J[i + 3 * n][j + 3 * n] = J_DD[i][j]
    return J



In [228]:
def S_alpha(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    n = number_of_Lks
    S_a = np.array([0 for i in range(n)])
    for i in range(n):
        res0 = - method[0] * S[i] * I[i]
        res1 = - method[1] * N_eff(i) * S[i] * I[i] / N[i]
        sum0 = 0
        for j in range(n):
            sum0 = sum0 + P(i, j) / N[i] * I_eff(I, j, t)
        res2 = - method[1] * S[i] * sum0
        S_a[i] = res0 + res1 + res2
    return S_a

def S_beta(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    return np.array([0 for i in range(n)])

def S_p(state, number_of_Lks, method, params, t):
    return S_beta(state, number_of_Lks, method, params, t)

def I_alpha(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    I_a = np.array([0 for i in range(n)])
    for i in range(n):
        res0 = method[0] * S[i] * I[i]
        res1 = method[1] * I_eff(I, i) * N_eff(i) * S[i] / N[i]
        sum0 = 0
        for j in range(n):
            sum0 = sum0 + P(i,j,t) *I_eff(I, j) / N[i]
        res2 =  sum0 * S[i] * method[1]
        I_a[i] = res0 + res1 + res2  
    return I_a

def I_beta(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    I = state[n:2*n,t]
    return - I

def I_p(state, number_of_Lks, method, params, t):
    return S_beta(state, number_of_Lks, method, params, t)

def R_alpha(state, number_of_Lks, method, params, t):
    return S_beta(state, number_of_Lks, method, params, t)

def R_beta(state, number_of_Lks, method, params, t):
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    n = number_of_Lks
    p = params[2]
    R_b = np.array([(1 - p) * I[i] for i in range(n)])
    return R_b

def R_p(state, number_of_Lks, method, params, t):
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    n = number_of_Lks
    beta = params[1]
    R = np.array([- beta * I[i] for i in range(n)])
    return R

def D_alpha(state, number_of_Lks, method, params, t):
    return S_beta(state, number_of_Lks, method, params, t)

def D_beta(state, number_of_Lks, method, params, t):
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    n = number_of_Lks
    beta = params[1]
    p = params[2]
    D_b = np.array([p * I[i] for i in range(n)])
    return D_b

def D_p(state, number_of_Lks, method, params, t):
    S = state[0:number_of_Lks,t]
    I = state[number_of_Lks:2*number_of_Lks,t]
    R = state[2*number_of_Lks:3*number_of_Lks,t]
    D = state[3*number_of_Lks:4*number_of_Lks,t]
    n = number_of_Lks
    beta = params[1]
    p = params[2]
    Dp = np.array([beta * I[i] for i in range(n)])
    return Dp
    

In [229]:
def Jacobian_F(state, solt, number_of_Lks, method, params, t):
    time_of_day = solt[t] - np.floor(solt[t])
    if method=='constant':
        X = [0.6,0.4]
    elif method=='heaviside':
        if time_of_day >=tzero[0] and time_of_day <= tzero[1]:
            X = [1,0]
        elif time_of_day>tzero[1] and time_of_day <= tzero[2]:
            X = [0,1]
    else:
        print("invalid Method")
        X = [0.6,0.4]
    Dp = D_p(state, number_of_Lks, X, params, t)
    D_b = D_beta(state, number_of_Lks, X, params, t)
    D_a = D_alpha(state, number_of_Lks, X, params, t)
    Rp = R_p(state, number_of_Lks, X, params, t)
    R_b = R_beta(state, number_of_Lks, X, params, t)
    R_a = R_alpha(state, number_of_Lks, X, params, t)
    Ip = I_p(state, number_of_Lks, X, params, t)
    I_b = I_beta(state, number_of_Lks, X, params, t)
    I_a = I_alpha(state, number_of_Lks, X, params, t)
    Sp = S_p(state, number_of_Lks, X, params, t)
    S_b = S_beta(state, number_of_Lks, X, params, t)
    S_a = S_alpha(state, number_of_Lks, X, params, t)
    a = np.concatenate((S_a, I_a, R_a, D_a), axis=0, out=None, dtype=None, casting="same_kind")
    b = np.concatenate((S_b, I_b, R_b, D_b), axis=0, out=None, dtype=None, casting="same_kind")
    p = np.concatenate((Sp, Ip, Rp, Dp), axis=0, out=None, dtype=None, casting="same_kind")
    res = np.vstack((a, b, p)).T
    
    return res



## Fehler Propagator

In [230]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def propagator_Matrix(state, solt, number_of_Lks, method, params, times = -1):
    n = number_of_Lks
    if times == -1:
        times = len(solt)
    ## finde h_j
    h = []
    dt = 0
    for j in range(times):
        if j+1<times:
            dt = solt[j+1] - solt[j]
        h.append(dt)
    ## finde A
    A = np.identity(n*4)
    AA = [A]
    for i in range(0, times):
        B = np.identity(n*4) + h[i] *Jacobian(state, solt, number_of_Lks, method, params, i) 
        A = AA.append(np.dot(B, AA[i]))
    ## finde F
    F = np.zeros((n*4, len(params)))
    FF = [F]
    for i in range(times):
        A_z = np.identity(n*4)
        B_i = Jacobian_F(state, solt, number_of_Lks, method, params, i)
        for j in range(i+1, times):
            B_z = AA[j]#np.identity(n*4) + h[j] *Jacobian(state, solt, number_of_Lks, method, params, j) 
            A_z = np.dot(B_z, A_z)
        F = F + np.dot(A_z, B_i)
        FF.append(F)
    res = [np.hstack((AA[i],FF[i])) for i in range(len(AA))]
    return res

def Kovarianz(Sigma,state, solt, number_of_Lks, method, params, times = -1):
    """
    liefer Kovarianz-Matrix des Systems bei Zeitpunkt solt[time]
    """
    result = []
    MM =  propagator_Matrix(state, solt, number_of_Lks, method, params, times)
    for i in range(times+1):
        M = MM[i]
        Kov = M.dot(Sigma.dot(M.T))
        if not is_pos_def(Kov):
            print("Matrix ist nicht positiv definit!")
        result.append(Kov)
    return result

def error(Sigma, lk, times):
    return np.sqrt(Sigma[times][lk][lk])

In [ ]:
Res = Kovarianz(Sigma, state, solt, number_of_Lks, method, params, times = 100)


### To do
 - **Zelle mit Res laufen**
 - **Plots**
 
